In [1]:
import pandas as pd
from string import punctuation
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# Part B

In [2]:
tweets = pd.read_csv('6ktweets.csv')

def remove_punctuation(item):
    for p in punctuation:
        item = item.lstrip().replace(p,'')
    return item

def conv_lowercase(x):
    return x.lower()

tweets['clean_tweets'] = tweets['tweets'].apply(remove_punctuation).apply(conv_lowercase)
#tweets

In [3]:
# nltk.download()
tweets['word_freq_list'] = tweets['clean_tweets'].apply(word_tokenize).apply(set).apply(list)

# Remove stop words
stop_words = set(stopwords.words('english'))
def remove_stopwords(s):
    return [w for w in s if not w in stop_words] 

tweets['word_freq_list'] = tweets['word_freq_list'].apply(remove_stopwords)

# Creating a list of all the words in the comments to count word frequency in the next step
count = []
for i in range(len(tweets)):
    count+=tweets['word_freq_list'][i]

In [4]:
# fetching the frequency of words
from nltk import FreqDist
word_freq = nltk.FreqDist(count)

In [5]:
#identified key issues

#trump issues
immigration = ['mexican', 'wall', 'immigrate', 'mexico', 'illegals', 'immigrants', 'immigrant']
impeach = ['impeachment', 'impeached', 'trumpimpeachment', 'impeachtrump']
foreign_relations = ['ukraine', 'isis', 'syria', 'china' ,'iran']

#biden issues
economy = ['job', 'jobs', 'money', 'taxes', 'taxpayer', 'economys', 'trumpeconomy']
education = [ 'college', 'school' ,'educate' ,'students' , 'student']
gun_control = ['control','weapons', 'weapon','guns', 'gun']

#replacements for trump and biden
trump = ['donaldtrump', 'realdonaldtrump',  'trump2020','trumps', 'donald' ]
biden = ['joebiden', 'joes','bidens', 'joe']

attributes = immigration + impeach+ foreign_relations + economy + trump + biden +education + gun_control

In [6]:
#replacing words with key issues
key_issues = pd.DataFrame()
key_issues['attribute'] = attributes
key_issues['general_attribute'] = ''

key_issues['general_attribute'][0:len(immigration)] = 'immigration'
key_issues['general_attribute'][len(immigration):len(immigration)+len(impeach)] = 'impeach'
key_issues['general_attribute'][len(immigration)+len(impeach):len(immigration)+len(impeach)+len(foreign_relations)] = 'foreign_relations'
key_issues['general_attribute'][len(immigration)+len(impeach)+len(foreign_relations):len(immigration)+len(impeach)+len(foreign_relations)+len(economy)] = 'economy'
key_issues['general_attribute'][len(immigration)+len(impeach)+len(foreign_relations)+ len(economy):len(immigration)+len(impeach)+len(foreign_relations)+len(economy)+len(trump)] = 'trump'
key_issues['general_attribute'][len(immigration)+len(impeach)+len(foreign_relations)+len(economy)+len(trump):len(immigration)+len(impeach)+len(foreign_relations)+len(economy)+len(trump)+len(biden)] = 'biden'
key_issues['general_attribute'][len(immigration)+len(impeach)+len(foreign_relations)+len(economy)+len(trump)+ len(biden):len(immigration)+len(impeach)+len(foreign_relations)+len(economy)+len(trump)+len(biden) + len(education)] = 'education'
key_issues['general_attribute'][len(immigration)+len(impeach)+len(foreign_relations)+len(economy)+len(trump)+ len(biden)+ len(gun_control):len(immigration)+len(impeach)+len(foreign_relations)+len(economy)+len(trump)+len(biden) + len(education)+len(gun_control)] = 'gun_control'
#key_issues

In [7]:
def word_to_attributes(s):
    s = " ".join(str(x) for x in s)
    for i in key_issues.index.values:
        s = s.replace(key_issues["attribute"][i].lower(),key_issues["general_attribute"][i].lower())
    return s

tweets['comments_attributes_replace'] = tweets['word_freq_list'].apply(word_to_attributes)


### C.To isolate a candidate and an issue, you have to write a parser that selects tweets that mention a candidate, takes a window around an issue, and chops off everything else, as shown in class.  

In [8]:
#making list of tweets that mention trump and list of tweets that mention biden
all_ = tweets['comments_attributes_replace'].apply(word_tokenize).apply(set).apply(list)
all_ = all_.tolist()
t =[]
b =[]
for i in all_: 
    for j in set(i):
        if j == 'trump':
            t.append(i)
        elif j == 'biden':
            b.append(i)             

In [9]:
#calculating total no of tweets that mention each of these key issues
economy =[]
immigration =[]
foreign_relations=[]
impeach=[]
for j in all_:
    for k in j:
        if k == 'economy':
            economy.append(j)
        if k == 'immigration':
            immigration.append(j)
        if k =='foreign_relations':
            foreign_relations.append(j)
        if k == 'impeach':
            impeach.append(j) 

In [10]:
#calculating total number of tweets that mention both key issues and trump
economy_trump =[]
immigration_trump =[]
foreign_relations_trump=[]
impeach_trump =[]

for j in t:
    for k in j:
        if k == 'economy':
            economy_trump.append(j)
        if k == 'immigration':
            immigration_trump.append(j)
        if k =='foreign_relations':
            foreign_relations_trump.append(j)
        if k == 'impeach':
            impeach_trump.append(j)           

In [11]:
#calculating total number of tweets that mention both key issues and biden
economy_biden =[]
immigration_biden =[]
foreign_relations_biden=[]
impeach_biden =[]

for c in b:
    for b1 in set(c):
        if b1 == 'economy':
            economy_biden.append(c)
            continue;
        if b1 == 'immigration':
            immigration_biden.append(c)
            continue; 
        if b1 =='foreign_relations':
            foreign_relations_biden.append(c) 
            continue;
        if b1 == 'impeach':
            impeach_biden.append(c)  
            

In [12]:
#lift analysis for trump and biden

#trump
lift_economy_trump = (len(tweets)*len(economy_trump))/(len(t)*len(economy))
lift_immigration_trump = (len(tweets)*len(immigration_trump))/(len(t)*len(immigration))
lift_foriegn_relations_trump = (len(tweets)*len(foreign_relations_trump))/(len(t)*len(foreign_relations))
lift_impeach_trump = (len(tweets)*len(impeach_trump))/(len(t)*len(impeach))

#biden
lift_economy_biden = (len(tweets)*len(economy_biden))/(len(b)*len(economy))
lift_immigration_biden = (len(tweets)*len(immigration_biden))/(len(b)*len(immigration))
lift_foriegn_relations_biden = (len(tweets)*len(foreign_relations_biden))/(len(b)*len(foreign_relations))
lift_impeach_biden = (len(tweets)*len(impeach_biden))/(len(b)*len(impeach))

In [13]:
data = [[lift_economy_trump, lift_immigration_trump, lift_foriegn_relations_trump, lift_impeach_trump], [lift_economy_biden, lift_immigration_biden, lift_foriegn_relations_biden,lift_impeach_biden]]
lift = pd.DataFrame(data, columns = ['Economy', 'Immigration', 'Foreign Relations', 'Impeach'])
lift.rename(index = {0: "Trump", 1:"Biden"},  inplace = True) 

In [14]:
lift

,Economy,Immigration,Foreign Relations,Impeach
Trump,0.677630,1.157184,1.317904,1.712633
Biden,1.794325,0.622407,2.489627,0.398340


In [15]:
#forming window around tweets that mention trump and issues

from nltk.tokenize.treebank import TreebankWordDetokenizer
impeach_sentiment_trump=[]
economy_sentiment_trump=[]
immigration_sentiment_trump=[]
foreign_relations_sentiment_trump=[]
for s in impeach_trump:
    for i in range(len(s)):
        if s[i] =='impeach':
            impeach_sentiment_trump.append(TreebankWordDetokenizer().detokenize(s[i-2:i+2]))
            
for s in economy_trump:
    for i in range(len(s)):
        if s[i] =='economy':
            economy_sentiment_trump.append(TreebankWordDetokenizer().detokenize(s[i-2:i+2]))

for s in immigration_trump:
    for i in range(len(s)):
        if s[i] =='immigration':
            immigration_sentiment_trump.append(TreebankWordDetokenizer().detokenize(s[i-2:i+2]))

for s in impeach_trump:
    for i in range(len(s)):
        if s[i] =='foreign_relations':
            foreign_relations_sentiment_trump.append(TreebankWordDetokenizer().detokenize(s[i-2:i+2]))

In [16]:
#forming window around tweets that mention biden and issues
impeach_sentiment_biden=[]
economy_sentiment_biden=[]
immigration_sentiment_biden=[]
foreign_relations_sentiment_biden=[]
for s in impeach_biden:
    for i in range(len(s)):
        if s[i] =='impeach':
            impeach_sentiment_biden.append(TreebankWordDetokenizer().detokenize(s[i-2:i+2]))
            
for s in economy_biden:
    for i in range(len(s)):
        if s[i] =='economy':
            economy_sentiment_biden.append(TreebankWordDetokenizer().detokenize(s[i-2:i+2]))

for s in immigration_biden:
    for i in range(len(s)):
        if s[i] =='immigration':
            immigration_sentiment_biden.append(TreebankWordDetokenizer().detokenize(s[i-2:i+2]))

for s in foreign_relations_biden:
    for i in range(len(s)):
        if s[i] =='foreign_relations':
            foreign_relations_sentiment_biden.append(TreebankWordDetokenizer().detokenize(s[i-2:i+2]))

In [17]:
#performing sentiment analysis 

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

def sentiment_analysis(list_):

    sia = SIA()
    r = []

    for i in list_:
        score_ = sia.polarity_scores(i)
        r.append(score_)
    sum_=0
    for i in range(len(r)):
        sum_ = sum_ + r[i]['pos']- r[i]['neg']
    return sum_ 

In [18]:
#getting lift scores for all key issues for each candidate

impeach_sentiment_t= sentiment_analysis(impeach_sentiment_trump)
economy_sentiment_t= sentiment_analysis(economy_sentiment_trump)
immigration_sentiment_t =sentiment_analysis(immigration_sentiment_trump)
foreign_relations_sentiment_t= sentiment_analysis(foreign_relations_sentiment_trump)

impeach_sentiment_b = sentiment_analysis(impeach_sentiment_biden)
economy_sentiment_b= sentiment_analysis(economy_sentiment_biden)
immigration_sentiment_b =sentiment_analysis(immigration_sentiment_biden)
foreign_relations_sentiment_b= sentiment_analysis(foreign_relations_sentiment_biden)


In [19]:
data = [[economy_sentiment_t, immigration_sentiment_t, foreign_relations_sentiment_t, impeach_sentiment_t], [economy_sentiment_b, immigration_sentiment_b, foreign_relations_sentiment_b,impeach_sentiment_b]]
sentiment = pd.DataFrame(data, columns = ['Economy', 'Immigration', 'Foreign Relations', 'Impeach'])
sentiment.rename(index = {0: "Trump", 1:"Biden"},  inplace = True) 

In [20]:
sentiment

,Economy,Immigration,Foreign Relations,Impeach
Trump,0.544,0.474,0.455,2.074
Biden,0.516,0.583,-0.888,0.444
